
# Hubmap Kaggle Competition

## Import needed libraries

In [ ]:
%cd ../input/thoplib
! pip install thop-0.0.31.post2005241907-py3-none-any.whl
%cd /kaggle/working

In [ ]:
import numpy as np
import pandas as pd
import rasterio
import numba, cv2, gc
import pathlib, sys, os, random, time
from datetime import datetime
from rasterio.windows import Window

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm

import albumentations as A

%cd ../input/pranet-for-hubmap
from PraNet.lib.PraNet_Res2Net import PraNet
from PraNet.utils.utils import clip_gradient, adjust_lr, AvgMeter
%cd /kaggle/working

In [ ]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as D
import torchvision.transforms as T
from torch.autograd import Variable

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Check GPU
print(DEVICE)

In [ ]:
def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seeds();

## Hyperparameters Config

In [ ]:
class HubmapConfig:

    EPOCHS = 20  # epoch number

    LR = 4e-4  # learning rate

    WEIGHT_DECAY = 1e-4  # weight decay

    BATCH_SIZE = 2 # training batch size

    CLIP = 0.5  # gradient clipping margin

    DECAY_RATE = 0.1  # decay rate of learning rate

    DECAY_EPOCH = 5  # every n epochs decay learning rate

    DATA_PATH = '../input/hubmap-kidney-segmentation'  # path to our dataset

    OUTPUT_PATH = './'  # path to output model

hubmapConfig = HubmapConfig()

## Load Dataset

In [ ]:
def rle2mask(rle, mask_shape):
    """
    decode the rle to image mask
    :param rle: a column ordered rle
    :param mask_shape: the shape of mask
    :return: a mask ndarray, 1-mask, 0-background
    """
    mask = np.zeros(np.prod(mask_shape), dtype=np.uint8)  # 1d mask array
    rle = np.array(rle.split()).astype(int)  # rle values to int
    starts = rle[::2]
    lengths = rle[1::2]
    for s, l in zip(starts, lengths):
        mask[s:s + l] = 1
    return mask.reshape(np.flip(mask_shape)).T  # flip because of the column-first ordered

def mask2rle(mask):
    """
    encode mask to rle
    :param mask: a mask adarray
    :return: column ordered rle
    """
    mask = mask.T.reshape(-1)  # make the mask 1d, column-first
    mask = np.pad(mask, 1)  # make sure that the 1d mask starts and ends with a 0
    starts = np.nonzero((~mask[:-1] & mask[1:]))[0]  # start points
    ends = np.nonzero(mask[:-1] & (~mask[1:]))[0]  # end points
    rle = np.empty(2 * starts.size, dtype=int)
    rle[0::2] = starts
    rle[1::2] = ends - starts
    rle = " ".join([str(elem) for elem in rle])
    return rle

@numba.njit()
def rle_numba(pixels):
    size = len(pixels)
    points = []
    if pixels[0] == 1: points.append(0)
    flag = True
    for i in range(1, size):
        if pixels[i] != pixels[i-1]:
            if flag:
                points.append(i+1)
                flag = False
            else:
                points.append(i+1 - points[-1])
                flag = True
    if pixels[-1] == 1: points.append(size-points[-1]+1)
    return points

def rle_numba_encode(image):
    pixels = image.flatten(order = 'F')
    points = rle_numba(pixels)
    return ' '.join(str(x) for x in points)

def make_grid(shape, window=256, min_overlap=32):
    """
    slice the image into N tiles, where N is the number of tiles
    :param shape: a tuple (x, y) represents the input shape
    :param window: the sliding window size
    :param min_overlap: overlap between tiles
    :return: Array of size (N, 4), 2nd axis represent slices: x1, x2, y1, y2
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)  # generate a array representing x1
    x1[-1] = x - window  # make the last tile can be a window size tile
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)  # generate a array representing y1
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx, ny, 4), dtype=np.int64)

    for i in range(nx):
        for j in range(ny):
            slices[i, j] = x1[i], x2[i], y1[j], y2[j]
    return slices.reshape(nx * ny, 4)

In [ ]:
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

class HubmapDataset(D.Dataset):

    def __init__(self, root_dir, transform,
                 window=256, overlap=32, threshold = 100):
        self.path = pathlib.Path(root_dir)
        self.overlap = overlap
        self.window = window
        self.transform = transform
        self.csv = pd.read_csv((self.path / 'train.csv').as_posix(),
                               index_col=[0])
        self.threshold = threshold

        self.x, self.y = [], []
        self.build_slices()
        self.len = len(self.x)
        self.as_tensor = T.Compose([
            T.ToTensor(),
            T.Normalize([0.625, 0.448, 0.688],
                        [0.131, 0.177, 0.101]),
        ])


    def build_slices(self):
        self.masks = []
        self.files = []
        self.slices = []
        for i, filename in enumerate(self.csv.index.values):
            filepath = (self.path /'train'/(filename+'.tiff')).as_posix()
            self.files.append(filepath)

            print('Transform', filename)
            with rasterio.open(filepath, transform = identity) as dataset:
                self.masks.append(rle2mask(self.csv.loc[filename, 'encoding'], dataset.shape))
                slices = make_grid(dataset.shape, window=self.window, min_overlap=self.overlap)

                for slc in tqdm(slices):
                    x1,x2,y1,y2 = slc
                    if self.masks[-1][x1:x2,y1:y2].sum() > self.threshold or np.random.randint(100) > 110:
                        self.slices.append([i,x1,x2,y1,y2])

                        image = dataset.read([1,2,3],
                            window=Window.from_slices((x1,x2),(y1,y2)))

#                         if image.std().mean() < 10:
#                             continue

                        # print(image.std().mean(), self.masks[-1][x1:x2,y1:y2].sum())
                        image = np.moveaxis(image, 0, -1)
                        self.x.append(image)
                        self.y.append(self.masks[-1][x1:x2,y1:y2])

    # get data operation
    def __getitem__(self, index):
        image, mask = self.x[index], self.y[index]
        augments = self.transform(image=image, mask=mask)
        return self.as_tensor(augments['image']), augments['mask'][None]

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [ ]:
WINDOW = 1024  # The tile size
MIN_OVERLAP = 64  # Overlapping between tiles
NEW_SIZE = 512  # Apply transformation to images and masks, this is the new size

In [ ]:
# Define
transform = A.Compose([
    A.Resize(NEW_SIZE,NEW_SIZE),
    A.OneOf([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
    ]),
    A.OneOf([
        A.RandomContrast(p=0.5),
        A.RandomGamma(p=0.5),
        A.RandomBrightness(p=0.5),
        A.ColorJitter(brightness=0.07, contrast=0.07,
                    saturation=0.1, hue=0.1, always_apply=False, p=0.5),
        A.CLAHE(p=0.5),
    ]),

    A.ShiftScaleRotate(),
])
   
hubmapDataset = HubmapDataset(hubmapConfig.DATA_PATH, window=WINDOW, overlap=MIN_OVERLAP, transform=transform)

In [ ]:
# Randomly show 3 images
random_list = random.sample(range(0, len(hubmapDataset)), 3)
for i in random_list:
    image, mask = hubmapDataset[i]
    plt.figure(figsize=(16, 8))
    plt.subplot(121)
    plt.imshow(mask[0], cmap='gray')
    plt.subplot(122)
    plt.imshow(image[0])
    plt.show()

In [ ]:
# Split dataset into train and val data
val_idx, train_idx = [], []
for i in range(len(hubmapDataset)):
    if i % 4 == 0:
        val_idx.append(i)
    else:
        train_idx.append(i)

In [ ]:
train_data = D.Subset(hubmapDataset, train_idx)
val_data = D.Subset(hubmapDataset, val_idx)

# Define training and validation data loader
train_loader = D.DataLoader(train_data, batch_size=hubmapConfig.BATCH_SIZE, shuffle=False, num_workers=2)
print(len(train_loader))
val_loader = D.DataLoader(val_data, batch_size=hubmapConfig.BATCH_SIZE, shuffle=False, num_workers=2)
print(len(val_loader))

## Define train and loss function

In [ ]:
class SoftDiceLoss(nn.Module):
    def __init__(self, smooth=1., dims=(-2,-1)):

        super(SoftDiceLoss, self).__init__()
        self.smooth = smooth
        self.dims = dims

    def forward(self, x, y):

        tp = (x * y).sum(self.dims)
        fp = (x * (1 - y)).sum(self.dims)
        fn = ((1 - x) * y).sum(self.dims)

        dc = (2 * tp + self.smooth) / (2 * tp + fp + fn + self.smooth)
        dc = dc.mean()

        return 1 - dc

bce_fn = nn.BCEWithLogitsLoss()
dice_fn = SoftDiceLoss()

def loss_fn(y_pred, y_true):
    y_true = y_true.type_as(y_pred)
    bce = bce_fn(y_pred, y_true)
    dice = dice_fn(y_pred.sigmoid(), y_true)
    return 0.8*bce+ 0.2*dice

In [ ]:
def train(train_loader, val_loader, model, optimizer, epoch, best_pred):
    # --- model save path ---
    save_path = hubmapConfig.OUTPUT_PATH
    os.makedirs(save_path, exist_ok=True)
    # --- start training ---
    model.train()
    # --- multi-scale training ---
    size_rates = [0.75, 1, 1.25]
    loss_record2, loss_record3, loss_record4, loss_record5 = AvgMeter(), AvgMeter(), AvgMeter(), AvgMeter()
    val_loss_record2, val_loss_record3, val_loss_record4, val_loss_record5 = AvgMeter(), AvgMeter(), AvgMeter(), AvgMeter()
    for i, pack in enumerate(train_loader, start=1):
        for rate in size_rates:
            optimizer.zero_grad()
            # --- data prepare ---
            images, gts = pack
            images = images.cuda()
            gts = gts.float().cuda()
            # --- rescale ---
            trainsize = int(round(NEW_SIZE * rate / 32) * 32)
            if rate != 1:
                images = F.upsample(images, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
                gts = F.upsample(gts, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            # --- forward ---
            lateral_map_5, lateral_map_4, lateral_map_3, lateral_map_2 = model(images)
            # --- loss function ---
            loss5 = loss_fn(lateral_map_5, gts)
            loss4 = loss_fn(lateral_map_4, gts)
            loss3 = loss_fn(lateral_map_3, gts)
            loss2 = loss_fn(lateral_map_2, gts)
            loss = loss2 + loss3 + loss4 + loss5
            # --- backward ---
            loss.backward()
            clip_gradient(optimizer, hubmapConfig.CLIP)
            optimizer.step()
            # --- recording loss and val loss---
            if rate == 1:
                loss_record2.update(loss2.data, hubmapConfig.BATCH_SIZE)
                loss_record3.update(loss3.data, hubmapConfig.BATCH_SIZE)
                loss_record4.update(loss4.data, hubmapConfig.BATCH_SIZE)
                loss_record5.update(loss5.data, hubmapConfig.BATCH_SIZE)
        # --- train visualization and validation---
        if i == 1 or i % 100 == 0 or i == len(train_loader):
            with torch.no_grad():
                for val_pack in val_loader:
                    val_images, val_gts = val_pack
                    val_images = val_images.cuda()
                    val_gts = val_gts.float().cuda()

                    val_lateral_map_5, val_lateral_map_4, val_lateral_map_3, val_lateral_map_2 = model(val_images)
                    val_loss5 = loss_fn(val_lateral_map_5, val_gts)
                    val_loss4 = loss_fn(val_lateral_map_4, val_gts)
                    val_loss3 = loss_fn(val_lateral_map_3, val_gts)
                    val_loss2 = loss_fn(val_lateral_map_2, val_gts)

                    val_loss_record2.update(val_loss2.data, 1)
                    val_loss_record3.update(val_loss3.data, 1)
                    val_loss_record4.update(val_loss4.data, 1)
                    val_loss_record5.update(val_loss5.data, 1)
            print('{} Epoch [{:03d}/{:03d}], Step [{:04d}/{:04d}], '
                  '[lateral-2: {:.4f}, lateral-3: {:0.4f}, lateral-4: {:0.4f}, lateral-5: {:0.4f}], '
                  '[val_lateral-2: {:.4f}, val_lateral-3: {:0.4f}, val_lateral-4: {:0.4f}, val_lateral-5: {:0.4f}]'.
                  format(datetime.now(), epoch, hubmapConfig.EPOCHS, i, len(train_loader),
                         loss_record2.show(), loss_record3.show(), loss_record4.show(), loss_record5.show(),
                         val_loss_record2.show(), val_loss_record3.show(), val_loss_record4.show(), val_loss_record5.show()))
            if val_loss_record2.show() < best_pred[-1]:
                best_pred.append(val_loss_record2.show())
                print("Best Val Loss: {}".format(val_loss_record2.show()))
                torch.save(model.state_dict(), os.path.join(save_path, 'model-best.pth'))
                print('[Saving Snapshot:]', os.path.join(save_path, 'model-best.pth'))

## Build model

In [ ]:
# torch.cuda.set_device(0)  # Set your gpu device
model = PraNet().cuda()  # Load model
model.load_state_dict(torch.load('../input/pranet-for-hubmap/PraNet/pretrained/PraNet-19.pth'))
params = model.parameters()  # Show model parameters
optimizer = torch.optim.AdamW(params, lr=hubmapConfig.LR, weight_decay=hubmapConfig.WEIGHT_DECAY)
# optimizer = torch.optim.Adam(params, lr=hubmapConfig.LR)

In [ ]:
print("#"*20, "Start Training", "#"*20)

best_pred = [10]
for epoch in range(1, hubmapConfig.EPOCHS+1):
    adjust_lr(optimizer, hubmapConfig.LR, epoch, hubmapConfig.DECAY_RATE, hubmapConfig.DECAY_EPOCH)
    train(train_loader, val_loader, model, optimizer, epoch, best_pred)

In [ ]:
del train_loader, val_loader, train_data, val_data, hubmapDataset
gc.collect()

In [ ]:
trfm1 = T.Compose([
    T.ToPILImage(),
    T.Resize(NEW_SIZE),
    T.ToTensor(),
    T.Normalize([0.625, 0.448, 0.688],
                [0.131, 0.177, 0.101]),
])

p = pathlib.Path(hubmapConfig.DATA_PATH)

subm = {}

model.load_state_dict(torch.load('model-best.pth'))
model.eval()

for i, filename in enumerate(p.glob('test/*.tiff')):
    dataset = rasterio.open(filename.as_posix(), transform = identity)
    slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
    preds = np.zeros(dataset.shape, dtype=np.uint8)
    for (x1,x2,y1,y2) in tqdm(slices):
        image = dataset.read([1,2,3],
                    window=Window.from_slices((x1,x2),(y1,y2)))
        image = np.moveaxis(image, 0, -1)
        image = trfm1(image)
        with torch.no_grad():
            image = image.to(DEVICE)[None]
            _, _, _, score = model(image)

            _, _, _, score2 = model(torch.flip(image, [0, 3]))
            score2 = torch.flip(score2, [3, 0])

            _, _, _, score3 = model(torch.flip(image, [1, 2]))
            score3 = torch.flip(score3, [2, 1])

            score_mean = (score + score2 + score3) / 3.0
            score_sigmoid = score_mean.sigmoid()
            score_sigmoid= F.upsample(score_sigmoid, size=(WINDOW, WINDOW), mode='bilinear', align_corners=False)
            score_sigmoid = score_sigmoid.cpu().numpy()

            preds[x1:x2,y1:y2] = (score_sigmoid > 0.5).astype(np.uint8)

    subm[i] = {'id':filename.stem, 'predicted': rle_numba_encode(preds)}
    del preds
    gc.collect();

In [ ]:
submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv('submission.csv', index=False)